In [11]:
!pip install pyarrow


In [12]:
import pandas as pd
import os

# Caminho da pasta com arquivos .xls
caminho_pasta = r"C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros"
saida=r"C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros_parquet"

def listar_todos_arquivos(diretorio_raiz, extensao=None):
    arquivos_encontrados = []
    for raiz, _, arquivos in os.walk(diretorio_raiz):
        for arquivo in arquivos:
            if extensao:
                if arquivo.lower().endswith(extensao.lower()):
                    arquivos_encontrados.append(os.path.join(raiz, arquivo))
            else:
                arquivos_encontrados.append(os.path.join(raiz, arquivo))
    return arquivos_encontrados

In [13]:
arquivos = listar_todos_arquivos(caminho_pasta,extensao='xls')

In [14]:
len(arquivos)

3737

In [15]:
# transforma os xls em parquet e unifica, para otimizar
dfs=[]

for arquivo in arquivos:
    try:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)

        # Lê o arquivo com pandas
        try: df_pd = pd.read_excel(caminho_arquivo,dtype=str)
        except: df_pd = pd.read_excel(caminho_arquivo)

        if not 'linha' in " ".join([x.lower() for x in df_pd.columns]):
            # Encontra a linha que contém "linha" para usar como cabeçalho
            idx_cabecalho = df_pd.apply(lambda row: row.astype(str).str.lower().str.contains("linha", case=False).any(), axis=1).idxmax()
            colunas = df_pd.iloc[idx_cabecalho].astype(str).str.strip().str.lower().tolist()

            # Lê os dados a partir da linha seguinte ao cabeçalho
            df_dados = df_pd.iloc[idx_cabecalho + 1:].reset_index(drop=True)
            df_dados.columns = colunas
        else:
            df_dados=df_pd

        df_dados['arquivo']=arquivo
        dfs.append(df_dados)
        caminho_parquet = os.path.join(saida, f"{arquivo.split('.')[0]}.parquet")
        df_dados.fillna("").to_parquet(caminho_parquet, index=False)
    except: print("Erro em ",arquivo)


Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\01062018.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\01092018.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\03122018_Dados para Transparência.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\06062018.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\07062018.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\08062018.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\10112018_Dados para Transparência.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\12112018_Dados para Transparência.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\14092018.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\2018\15102018_Dados para Transparência.xls
Erro em  C:\Users\franc\OneDrive\Documentos\TCC\dados\passag

In [16]:
#le parquets para pegar as colunas únicas e começar as transformações
# Lista de arquivos .parquet na pasta
arquivos_parquet = listar_todos_arquivos(caminho_pasta,extensao='parquet')
# print(len(arquivos_parquet), len(arquivos), len(dfs))


In [17]:
# Lista para armazenar as colunas de cada dataframe
colunas_df = []
dfs=[]

# Lê os arquivos .parquet e armazena as colunas de cada dataframe
for arquivo in arquivos_parquet:
    caminho_arquivo = os.path.join(saida, arquivo)
    # Lê o parquet no pandas
    df = pd.read_parquet(caminho_arquivo)
    colunas_df.append([x.lower() for x in df.columns])
    dfs.append(df)

#para visualização das colunas únicas
colunas_df = [list(x) for x in set(tuple(x) for x in colunas_df)]
lista=[]
for x in colunas_df: lista+= x

lista=list(set(lista))
lista.sort()
print("Colunas únicas:" , lista)

Colunas únicas: ['area', 'arquivo', 'data', 'empresa', 'grupo', 'linha', 'lote', 'passageiros com gratuidade', 'passageiros com gratuidade\n\n(l)', 'passageiros com gratuidade est', 'passageiros com gratuidade estudante\n\n(m)', 'passageiros com gratuidade estudante\n\n(n)', 'passageiros comum e vt', 'passageiros int ônibus->ônibus', 'passageiros integrações ônibus -> ônibus\n\n(k)', 'passageiros pagantes', 'passageiros pagantes\n\n(j) = (a+b+c+d+e+f+g+h+i)', 'passageiros pagantes bilhete único comum\n\n(b)', 'passageiros pagantes bilhete único comum mensal\n\n(c)', 'passageiros pagantes bilhete único estudante\n\n(d) ', 'passageiros pagantes bilhete único estudante mensal\n\n(e) ', 'passageiros pagantes bilhete único vale transporte\n\n(f)', 'passageiros pagantes bilhete único vale transporte mensal\n\n(g)', 'passageiros pagantes em dinheiro\n\n(a)', 'passageiros pagantes integrações metrô e cptm\n\n(h)', 'passageiros pagantes integrações metrô e cptm mensal\n\n(i)', 'passageiros pagt

In [18]:
renaming_dict={'area':"area",
 'arquivo':"arquivo",
 'data':"data",
 'empresa':"empresa",
 'grupo':"grupo",
 'linha':"linha",
 'lote':"lote",
 'passageiros com gratuidade':"passageiros_gratuitos",
 'passageiros com gratuidade\n\n(l)':"passageiros_gratuitos",
 'passageiros com gratuidade est':"passageiros_gratuitos_estudante",
 'passageiros com gratuidade estudante\n\n(m)':"passageiros_gratuitos_estudante",
 'passageiros com gratuidade estudante\n\n(n)':"passageiros_gratuitos_estudante",
 'passageiros comum e vt':"passageiros_comum_vt",
 'passageiros int ônibus->ônibus':"passageiros_integracao_onibus",
 'passageiros integrações ônibus -> ônibus\n\n(k)':"passageiros_integracao_onibus",
 'passageiros pagantes':"passageiros_pagantes",
 'passageiros pagantes\n\n(j) = (a+b+c+d+e+f+g+h+i)':"passageiros_pagantes",
 'passageiros pagantes bilhete único comum\n\n(b)':"passageiros_comum",
 'passageiros pagantes bilhete único comum mensal\n\n(c)':"passageiros_comum_mensal",
 'passageiros pagantes bilhete único estudante\n\n(d) ':"passageiros_estudante",
 'passageiros pagantes bilhete único estudante mensal\n\n(e) ':"passageiros_estudante_mensal",
 'passageiros pagantes bilhete único vale transporte\n\n(f)':"passageiros_vale_transporte",
 'passageiros pagantes bilhete único vale transporte mensal\n\n(g)':"passageiros_vale_transporte_mensal",
 'passageiros pagantes em dinheiro\n\n(a)':"passageiros_dinheiro",
 'passageiros pagantes integrações metrô e cptm\n\n(h)':"passageiros_integracao_metro_cptm",
 'passageiros pagantes integrações metrô e cptm mensal\n\n(i)':"passageiros_integracao_metro_cptm_mensal",
 'passageiros pagtes bu vt':"passageiros_comum_vt",
 'passageiros pagtes comum':"passageiros_comum",
 'passageiros pagtes em dinheiro':"passageiros_dinheiro",
 'passageiros pagtes estudante':"passageiros_estudante",
 'passageiros pagtes int m/cptm':"passageiros_integracao_metro_cptm",
 'passageiros pgts bu comum m':"passageiros_comum_mensal",
 'passageiros pgts bu est mensal':"passageiros_estudante_mensal",
 'passageiros pgts bu vt mensal':"passageiros_vale_transporte_mensal",
 'passageiros pgts int m/cptm m':"passageiros_integracao_metro_cptm_mensal",
 'soma de passageiros com gratuidade\n\n(l)':"passageiros_gratuitos",
 'soma de passageiros com gratuidade estudante\n\n(m)':"passageiros_gratuitos_estudante",
 'soma de passageiros integrações ônibus -> ônibus\n\n(k)':"passageiros_integracao_onibus",
 'soma de passageiros pagantes\n\n(j) = (a+b+c+d+e+f+g+h+i)':"passageiros_pagantes",
 'soma de passageiros pagantes bilhete único comum\n\n(b)':"passageiros_comum",
 'soma de passageiros pagantes bilhete único comum mensal\n\n(c)':"passageiros_comum_mensal",
 'soma de passageiros pagantes bilhete único estudante\n\n(d) ':"passageiros_estudante",
 'soma de passageiros pagantes bilhete único estudante mensal\n\n(e) ':"passageiros_estudante_mensal",
 'soma de passageiros pagantes bilhete único vale transporte\n\n(f)':"passageiros_vale_transporte",
 'soma de passageiros pagantes bilhete único vale transporte mensal\n\n(g)':"passageiros_vale_transporte_mensal",
 'soma de passageiros pagantes em dinheiro\n\n(a)':"passageiros_dinheiro",
 'soma de passageiros pagantes integrações metrô e cptm\n\n(h)':"passageiros_integracao_metro_cptm",
 'soma de passageiros pagantes integrações metrô e cptm mensal\n\n(i)':"passageiros_integracao_metro_cptm_mensal",
 'soma de total passageiros transportados\n\n(n) = (j + k + l + m)':'passageiros_total',
 'tipo':"tipo",
 'tot passageiros transportados':'passageiros_total',
 'total passageiros transportados\n\n(m) = (j + k + l )':'passageiros_total',
 'total passageiros transportados\n\n(m) = (j + k + l)':'passageiros_total',
 'total passageiros transportados\n\n(m) = (j + k + l+n)':'passageiros_total',
 'total passageiros transportados\n\n(n) = (j + k + l + m)':'passageiros_total'
 }

In [20]:
df_unificado=pd.DataFrame()
for df in dfs:
    df.columns=[x.lower() for x in df.columns]
    df=df.rename(columns=renaming_dict)
    df_unificado=pd.concat([df_unificado,df])

In [ ]:
df_unificado['data'] = pd.to_datetime(df_unificado['data'], errors='coerce')
df_unificado['route_id']=df_unificado['linha'].fillna("").apply(lambda x: x.split(" ")[0].replace("-","").title())

In [27]:
for c in [x for x in df_unificado.columns if 'passageiro' in x]: print(c);df_unificado[c]=df_unificado[c].fillna(0).str.replace("","0").astype('Int64') 

passageiros_dinheiro
passageiros_comum
passageiros_comum_mensal
passageiros_estudante
passageiros_estudante_mensal
passageiros_vale_transporte
passageiros_vale_transporte_mensal
passageiros_integracao_metro_cptm
passageiros_integracao_metro_cptm_mensal
passageiros_pagantes
passageiros_integracao_onibus
passageiros_gratuitos
passageiros_total
passageiros_gratuitos_estudante
passageiros_comum_vt


In [28]:
colunas_agrupar=['passageiros_comum_mensal',
       'passageiros_estudante_mensal',
       'passageiros_vale_transporte_mensal',
       'passageiros_integracao_metro_cptm',
       'passageiros_integracao_metro_cptm_mensal',
       'passageiros_integracao_onibus', 
       'passageiros_gratuitos_estudante',
       'passageiros_comum_vt', ]

df_unificado['passageiros_vale_transporte']+= df_unificado['passageiros_comum_vt'] + df_unificado['passageiros_vale_transporte_mensal']
df_unificado['passageiros_integracao']= df_unificado['passageiros_integracao_metro_cptm'] + df_unificado['passageiros_integracao_metro_cptm_mensal'] + df_unificado['passageiros_integracao_onibus']
df_unificado['passageiros_comum']+=df_unificado['passageiros_comum_mensal']
df_unificado['passageiros_estudante']+=df_unificado['passageiros_estudante_mensal']

In [29]:
df_unificado.to_parquet('dados/passageiros_unificado.parquet', index=False)

In [30]:
df_unificado.shape

(4937029, 25)